In [1]:
import numpy as np
import math
import plotly.graph_objects as go


In [41]:
t0=0
T=1500

a=0.5
l=0.1

x_m_init = a
y_m_init = l/2

x_p_init = a
y_p_init = -l/2

v_x_m_init, v_y_m_init, v_x_p_init, v_y_p_init = 0,0,0,0

q_m = -10**(-7)
q_p = 10**(-7)
Q_p = 1

mass = 0.1

lambda_init = 1
M=400

a_11=complex(0.5,0.5)

tau=(T-t0)/M

In [42]:
t=np.linspace(t0,T,M+1)
y=np.zeros((M+1,9))

y[0,:] = x_m_init,y_m_init,x_p_init,y_p_init, v_x_m_init, v_y_m_init,v_x_p_init,v_y_p_init,lambda_init

In [44]:
def func(y):
    f_vector = np.zeros((9,1))
    f_vector[0]=y[4]
    f_vector[1]=y[5]
    f_vector[2]=y[6]
    f_vector[3]=y[7]
    f_vector[4]=-abs(q_m)*Q_p*y[0]/mass/(y[0]**2+y[1]**2)**(3/2) - 2/mass*y[8]*(y[0]-y[2])
    f_vector[5]=-abs(q_m)*Q_p*y[1]/mass/(y[0]**2+y[1]**2)**(3/2) - 2/mass*y[8]*(y[1]-y[3])
    f_vector[6]=abs(q_p)*Q_p*y[2]/mass/(y[2]**2+y[3]**2)**(3/2) + 2/mass*y[8]*(y[0]-y[2])
    f_vector[7]=abs(q_p)*Q_p*y[3]/mass/(y[2]**2+y[3]**2)**(3/2) + 2/mass*y[8]*(y[1]-y[3])
    f_vector[8]=l**2-(y[0]-y[2])**2 - (y[1]-y[3])**2
    return f_vector

def func_y(y):
    f_y_matrix = np.zeros((9,9))
    f_y_matrix[0,4]=1
    f_y_matrix[1,5]=1
    f_y_matrix[2,6]=1
    f_y_matrix[3,7]=1
    
    f_y_matrix[4,0] =-abs(q_m)*Q_p/mass*(y[1]**2-2*y[0]**2)/(y[0]**2+y[1]**2)**(5/2) - 2/mass*y[8]
    f_y_matrix[4,1]= -abs(q_m)*Q_p/mass*(-3*y[0]*y[1])/(y[0]**2+y[1]**2)**(5/2) 
    f_y_matrix[4,2] = 2/mass*y[8]
    f_y_matrix[4,8] = -2/mass*(y[0]-y[2])
    
    f_y_matrix[5,0] = -abs(q_m)*Q_p/mass*(-3*y[0]*y[1])/(y[0]**2+y[1]**2)**(5/2)
    f_y_matrix[5,1] = -abs(q_m)*Q_p/mass*(y[0]**2-2*y[1]**2)/(y[0]**2+y[1]**2)**(5/2)-2/mass*y[8]
    f_y_matrix[5,3] = 2/mass*y[8]
    f_y_matrix[5,8]=-2/mass*(y[1]-y[3])
    
    f_y_matrix[6,0]=2/mass*y[8]
    f_y_matrix[6,2]=abs(q_p)*Q_p/mass*(y[3]**2 -2*y[2]**2)/(y[2]**2+y[3]**2)**(5/2)-2/mass*y[8]
    f_y_matrix[6,2]=abs(q_p)*Q_p/mass*(-3*y[2]*y[3])/(y[2]**2+y[3]**2)**(5/2)
    f_y_matrix[6,8] = 2/mass*(y[0]-y[2])
   
    f_y_matrix[7,1]=2/mass*y[8]
    f_y_matrix[7,2]=abs(q_p)*Q_p/mass*(-3*y[2]*y[3])/(y[3]**2+y[2]**2)**(5/2)
    f_y_matrix[7,3]=abs(q_p)*Q_p/mass*(y[2]**2-2*y[3]**2)/(y[3]**2+y[2]**2)**(5/2)-2/mass*y[8]
    f_y_matrix[7,8]=2/mass*(y[1]-y[3])
    
    f_y_matrix[8,0]=-2*(y[0]-y[2])
    f_y_matrix[8,1]=-2*(y[1]-y[3])
    f_y_matrix[8,2]=2*(y[0]-y[2])
    f_y_matrix[8,3]=2*(y[1]-y[3])

    return f_y_matrix

def D(N):
    d_matrix = np.eye(N)
    d_matrix[N-1,N-1]=0.001
    return d_matrix
    
    

In [45]:
for m in range(M):
    #w = np.dot(np.linalg.inv(D(9) - a_11*tau*func_y(y[m,:])),func(y[m,:]))
    w=np.linalg.solve(D(9) - a_11*tau*func_y(y[m,:]),func(y[m,:]))
    for i in range(9):
        
        y[m+1,i] = y[m,i] + tau*w[i].real

In [49]:
fig = go.Figure(
    data=[go.Scatter(x=[y[0,0],y[0,2]], y=[y[0,1],y[0,3]])],
    layout=go.Layout(
        xaxis=dict(range=[0, 0.7], autorange=False),
        yaxis=dict(range=[-0.1, 0.1], autorange=False),
        title="Start Title",
        updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=[y[k,0],y[k,2]],y=[y[k,1],y[k,3]], name='solution',
            mode="markers+lines",
            marker=dict(color="blue", size=8))])

        for k in range(M+1)]

)
fig.show()

In [46]:
fgr=go.Figure()
len = np.zeros(M+1)
len[:]=(y[:,0]-y[:,2])**2 + (y[:,1]-y[:,3])**2
fgr.add_trace(go.Scatter(x=t,y=len[:]))